In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

from pathlib import Path

# Feature detectors

In [ ]:
from pixelpoint.feature_detection import orb_sift_extract_keypoints_and_descriptors, orb_sift_draw_keypoints

In [ ]:
current_dir = Path.cwd()

data_dir = current_dir / 'data'
img_1_path = data_dir / 'cam2_1.jpg'
img_2_path = data_dir / 'cam1_1.jpg'

## ORB detector

In [ ]:
(kp1, _), (kp2, _) = orb_sift_extract_keypoints_and_descriptors(img_1_path, img_2_path, detector_type='ORB')

print(f"Keypoints in image 1: {len(kp1)}")
print(f"Keypoints in image 2: {len(kp2)}")

In [ ]:
orb_sift_draw_keypoints(img_1_path, kp1, img_2_path, kp2)

## SIFT detector

In [ ]:
(kp1, _), (kp2, _) = orb_sift_extract_keypoints_and_descriptors(img_1_path, img_2_path, detector_type='SIFT')

print(f"Keypoints in image 1: {len(kp1)}")
print(f"Keypoints in image 2: {len(kp2)}")

In [ ]:
orb_sift_draw_keypoints(img_1_path, kp1, img_2_path, kp2)

## SuperPoint

In [ ]:
from transformers import AutoImageProcessor, SuperPointForKeypointDetection
import torch
from PIL import Image

In [ ]:
from pixelpoint.feature_detection import load_images, process_images, superpoint_detect_keypoints, superpoint_visualize_keypoints

In [ ]:
image_paths = [img_1_path, img_2_path]
images = load_images(image_paths)

In [ ]:
processor = AutoImageProcessor.from_pretrained("magic-leap-community/superpoint")
model = SuperPointForKeypointDetection.from_pretrained("magic-leap-community/superpoint")

In [ ]:
model

In [ ]:
image_results = superpoint_detect_keypoints(images, model, processor)
superpoint_visualize_keypoints(image_results)

## Summary

1. Использовать SIFT не будем, т.к. скорость его работы низка, да и как видно, при матчинге будет очень много несоответствий.  
2. ORB тоже показывает неудовлетворительные результаты, т.к. как видно он не отмечает все точки на объекте равномерно.  
3. Метод SuperPoint (на основе нейронной сети) работает значительно лучше - равномерность точек по объекту. Вдобавок к алгоритму поиска ключевых точек SuperPoint существует алгоритм матчинга (от тех же исследователей/разработчиков) который является логичным продолжением этого - [SuperGlue](https://github.com/magicleap/SuperGluePretrainedNetwork/tree/master)

Оставшиеся задачи:
1. Попробовать другие детекторы на основе DL (D2Net, R2D2)
2. Применить последовательно к SuperPoint алгоритм SuperGlue для матчинга точек
3. Также остается идея для использования сегментации чтобы отделить сам объект от фона и уже только на нем искать характеристические точки